In [ ]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=46896 sha256=30d40c5194927672d71fd756f269f6d742ee57ed7e2a5405078520070250ec60
  Stored in directory: /tmp/pip-ephem-wheel-cache-w5lz57tm/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [ ]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import get_trained_rnn_model, evaluate_trained_rnn, across_suject_rnn_performance


In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/RNN/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for RNN training
epochs = 200
batch_size = 2
es_patience = 5
#number of permutations to use for training
nsets_training = 20
model_dict = {'fe_layers':1, 'fe_activation':'tanh'}

nreps = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']



for src_subject_id in range(1,nsubjects+1):

    print('---TRUE DATA---')
    results_df = across_suject_rnn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                               exclude, score_list,  nsets_training, epochs = epochs, batch_size = batch_size,\
                                               es_patience = es_patience, mv = None, permute = False)

    results_df['Shuffled'] = False

    print('---PERMUTED DATA---')
    perm_results_df = across_suject_rnn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                               exclude, score_list,  nsets_training, epochs = epochs, batch_size = batch_size,\
                                               es_patience = es_patience, mv = None, permute = True)

    perm_results_df['Shuffled'] = True

    #concatenate

    results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
    results_df['Train_Subject'] = src_subject_id

    #save results to file
    results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')



Streaming output truncated to the last 5000 lines.
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/24
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/25
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/26
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/27
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/28
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/29
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/30
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/31
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/32
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/34
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/35
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/36
Evaluate Model
S

In [ ]:
results_df.groupby(['Shuffled','Type']).mean()

Rep  Test_Subject  ...  accuracy_score  Train_Subject
Shuffled Type                      ...                               
False    Test   1.5           1.5  ...        0.461552            3.0
         Train  1.5           3.0  ...        0.995461            3.0
True     Test   1.5           1.5  ...        0.161126            3.0
         Train  1.5           3.0  ...        0.842895            3.0

[4 rows x 10 columns]